In [11]:
import pandas as pd 
import numpy as np

from difflib import get_close_matches

import re

In [12]:
df_final = pd.read_csv('../../Data/final_movies.csv')
df_budget= pd.read_csv('../../Data/movie_budgets_name_fixed (2).csv')

In [13]:
def reformat_title(title):
    formated_str = re.sub(r"[^a-zA-Z0-9:' .]", '', title).strip()
    formated_str = re.sub(' +', ' ', formated_str)
    return formated_str.replace(' ','').lower()

In [14]:
df_final['Reformated name'] = df_final['Name'].apply(reformat_title)
df_budget['Reformated name'] = df_budget['Movie Name'].apply(reformat_title)

In [15]:
count = 0
count = 0

title_list = df_final['Reformated name']

for i in range(len(df_budget['Reformated name'])):
    match = get_close_matches(df_budget['Reformated name'][i], title_list, n = 1, cutoff=0.9)
    if len(match) > 0:
        df_budget.loc[i, 'Reformated name'] = match[0]
        count += 1

print('\n',count)


 5208


In [16]:
df_budget

,Release Date,Movie Name,Production Budget,Domestic Gross,Worldwide Gross,Reformated name
0,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$533,200,000","$936,662,225","$2,056,444,271",starwarsep.vii:theforceawakens
1,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$684,075,767","$2,317,514,386",avatarthewayofwater
2,"Jun 28, 2023",Indiana Jones and the Dial of Destiny,"$402,300,000","$174,480,468","$383,963,057",indianajonesandthedialofdestiny
3,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,748,242,781",avengersendgame
4,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802",piratesofthecaribbeanonstrangertides
...,...,...,...,...,...,...
6535,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",following
6536,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",returntothelandofwonders
6537,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,aplaguesopleasant
6538,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041",mydatewithdrew


In [17]:
def extract_year(year_str):
    if isinstance(year_str, str):
        match = re.search(r'\b(19|20)\d{2}\b', year_str)
        if match:
            return match.group()
        match = re.search(r'(19|20)\d{2}', year_str)
        if match:
            return match.group()
        if year_str.lower() == 'unknown' or year_str.lower() == 'nan':
            return np.nan
    return np.nan 

In [18]:
df_final['Year'] = df_final['Year'].apply(extract_year)
df_budget['Year'] = df_budget['Release Date'].apply(extract_year)

In [19]:
df_fulldata = pd.merge(df_final, df_budget, on=['Reformated name'], how='inner')
df_fulldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5241 entries, 0 to 5240
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               5241 non-null   object
 1   Year_x             5179 non-null   object
 2   CriticScore        4233 non-null   object
 3   UserScore          4632 non-null   object
 4   Link               5241 non-null   object
 5   PlatformReleased   5241 non-null   object
 6   Cast               5208 non-null   object
 7   Director           5204 non-null   object
 8   Genre              5194 non-null   object
 9   Rating             4106 non-null   object
 10  Runtime            5167 non-null   object
 11  Studio             5193 non-null   object
 12  Reformated name    5241 non-null   object
 13  Release Date       5241 non-null   object
 14  Movie Name         5241 non-null   object
 15  Production Budget  5241 non-null   object
 16  Domestic Gross     5241 non-null   object


In [20]:
df_fulldata.rename(columns={'Name' : 'Title'}, inplace=True)
df_fulldata = df_fulldata.drop('Year_x', axis=1)
df_fulldata = df_fulldata.drop('Year_y', axis=1)
df_fulldata = df_fulldata.drop('Movie Name', axis=1)
df_fulldata = df_fulldata.drop('Reformated name', axis=1)

df_fulldata.drop_duplicates(subset='Title', ignore_index=True, inplace=True)

df_fulldata.to_csv('../../Data/full_data.csv', index=False)

In [25]:
df_result_1 = pd.read_csv('../../Data/results.csv')
df_result_2 = pd.read_csv('../../Data/updated_output.csv')

In [26]:
df_result_2 = df_result_2.drop(columns='Identifier', axis = 1)
df_result_2.rename(columns={'Movie Title' : 'Title'}, inplace=True)
df_result_2.rename(columns={'metascore' : 'Metascore'}, inplace=True)
df_result_2.rename(columns={'userscore' : 'Userscore'}, inplace=True)


In [27]:
df_result = pd.concat([df_result_1, df_result_2], ignore_index=True, sort=False)
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5379 entries, 0 to 5378
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      5379 non-null   object
 1   Metascore  4742 non-null   object
 2   Userscore  4742 non-null   object
dtypes: object(3)
memory usage: 126.2+ KB


In [28]:
df_result['Formated name'] = df_result['Title'].apply(reformat_title)
df_fulldata['Formated name'] = df_fulldata['Title'].apply(reformat_title)
df_result['Formated name'] = df_result['Title'].apply(reformat_title)

In [29]:
count = 0
count = 0

title_list = df_fulldata['Formated name']

for i in range(len(df_result['Formated name'])):
    match = get_close_matches(df_result['Formated name'][i], title_list, n = 1, cutoff=0.9)
    if len(match) > 0:
        df_result.loc[i, 'Formated name'] = match[0]
        count += 1

print('\n',count)


 5355


In [30]:
df_fulldata

,Title,CriticScore,UserScore,Link,PlatformReleased,Cast,Director,Genre,Rating,Runtime,Studio,Release Date,Production Budget,Domestic Gross,Worldwide Gross,Formated name
0,LAConfidential,99%,94%,https://www.rottentomatoes.com/m/la_confidential,Cinema,"Kevin Spacey, Russell Crowe, Guy Pearce, James...",Curtis Hanson,"Crime, Drama",R,2h 16m,Warner Home Vídeo,"Sep 19, 1997","$35,000,000","$64,604,977","$126,204,977",laconfidential
1,TheGodfather,97%,98%,https://www.rottentomatoes.com/m/the_godfather,Cinema,"Marlon Brando, Al Pacino, James Caan, Richard ...",Francis Ford Coppola,"Crime, Drama",R,2h 57m,Paramount Pictures,"Mar 15, 1972","$7,000,000","$136,479,994","$270,007,394",thegodfather
2,Casablanca,99%,95%,https://www.rottentomatoes.com/m/1003707-casab...,Cinema,"Humphrey Bogart, Ingrid Bergman, Paul Henreid,...",Michael Curtiz,Drama,PG,1h 42m,Warner Bros. Pictures,"Jan 23, 1943","$1,039,000","$10,462,500","$10,567,179",casablanca
3,Parasite,99%,90%,https://www.rottentomatoes.com/m/parasite_2019,Cinema,"Song Kang-ho, Lee Sun-kyun, Jo Yeo-jeong, Choi...",Bong Joon Ho,"Comedy, Mystery & Thriller, Drama",R,2h 12m,Neon,"May 30, 2019","$11,800,000","$53,369,745","$253,267,858",parasite
4,SchindlersList,98%,97%,https://www.rottentomatoes.com/m/schindlers_list,Cinema,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",Steven Spielberg,"History, Drama",R,3h 15m,Universal Pictures,"Dec 15, 1993","$25,000,000","$96,898,818","$322,197,130",schindlerslist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5090,AntarcticEdge70South,57%,50%,https://www.rottentomatoes.com/m/antarctic_edg...,Cinema,"Dena Seidel, Richard Ludescher, Dena Seidel",Dena Seidel,"Documentary, Adventure",NaN,1h 12m,First Run,"Apr 17, 2015","$150,000","$7,193","$7,193",antarcticedge70south
5091,8DaystoHell,NaN,NaN,https://www.rottentomatoes.com/m/8_days_to_hell,Cinema,"Eric Roberts, Shane Woodson, Drew Hale, David ...",Shane Woodson,Horror,NaN,1h 25m,Pegasus Flying Films,"Jul 30, 2021","$100,000",$0,$0,8daystohell
5092,20Dates,35%,47%,https://www.rottentomatoes.com/m/20_dates,Cinema,"Myles Berkowitz, Elisabeth Wagner, Richard Arl...",Myles Berkowitz,"Comedy, Romance",R,1h 32m,Fox,"Feb 26, 1999","$66,000","$541,636","$602,920",20dates
5093,Happy40th,NaN,NaN,https://www.rottentomatoes.com/m/happy_40th,Cinema,"Fernando Acosta, Jenni Blong, Robyn Cohen, Mad...",Madoka Raine,Drama,NaN,1h 40m,English,Unknown,"$35,000",$0,$0,happy40th


In [31]:
df_fulldata = pd.merge(df_fulldata, df_result, on=['Formated name'], how='inner')
df_fulldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5373 entries, 0 to 5372
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title_x            5373 non-null   object
 1   CriticScore        4354 non-null   object
 2   UserScore          4760 non-null   object
 3   Link               5373 non-null   object
 4   PlatformReleased   5373 non-null   object
 5   Cast               5340 non-null   object
 6   Director           5335 non-null   object
 7   Genre              5326 non-null   object
 8   Rating             4222 non-null   object
 9   Runtime            5300 non-null   object
 10  Studio             5325 non-null   object
 11  Release Date       5373 non-null   object
 12  Production Budget  5373 non-null   object
 13  Domestic Gross     5373 non-null   object
 14  Worldwide Gross    5373 non-null   object
 15  Formated name      5373 non-null   object
 16  Title_y            5373 non-null   object


In [32]:
df_fulldata = df_fulldata.drop(columns='Title_y', axis=1)
df_fulldata.rename(columns={'Title_x' : 'Title'}, inplace=True)
df_fulldata.drop_duplicates(subset='Title', ignore_index=True, inplace=True)

In [33]:
df_fulldata.to_csv("../../Data/full_data.csv", index= False)